### 이슈 발생
- 데이터를 join하는 과정에서 불필요하게 많은 컬럼들이 조인이 되었던 문제점을 발견함(250703)
- 기존과 똑같이 데이터를 추출하되, join 과정을 줄여서 train_r05(행: 24만개), test_r05 데이터를 구축함. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 기본 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc

# 그래프 기본 테마 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 복잡한 통계 처리를 위한 라이브러리
from scipy import stats
from sklearn.preprocessing import LabelEncoder


### Train 데이터 생성

In [3]:
df107 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/1.회원정보/201807_train_회원정보.parquet')
df108 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/1.회원정보/201808_train_회원정보.parquet')
df109 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/1.회원정보/201809_train_회원정보.parquet')
df110 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/1.회원정보/201810_train_회원정보.parquet')
df111 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/1.회원정보/201811_train_회원정보.parquet')
df112 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/1.회원정보/201812_train_회원정보.parquet')

df107.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 78 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   기준년월                400000 non-null  int64  
 1   ID                  400000 non-null  object 
 2   남녀구분코드              400000 non-null  int64  
 3   연령                  400000 non-null  object 
 4   Segment             400000 non-null  object 
 5   회원여부_이용가능           400000 non-null  int64  
 6   회원여부_이용가능_CA        400000 non-null  int64  
 7   회원여부_이용가능_카드론       400000 non-null  int64  
 8   소지여부_신용             400000 non-null  int64  
 9   소지카드수_유효_신용         400000 non-null  int64  
 10  소지카드수_이용가능_신용       400000 non-null  int64  
 11  입회일자_신용             400000 non-null  int64  
 12  입회경과개월수_신용          400000 non-null  int64  
 13  회원여부_연체             400000 non-null  int64  
 14  이용거절여부_카드론          400000 non-null  int64  
 15  동의여부_한도증액안내         400000 non-nul

In [4]:
# 1번 데이터 병합
df1 = pd.concat([df107, df108, df109, df110, df111, df112], ignore_index=True)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400000 entries, 0 to 2399999
Data columns (total 78 columns):
 #   Column              Dtype  
---  ------              -----  
 0   기준년월                int64  
 1   ID                  object 
 2   남녀구분코드              int64  
 3   연령                  object 
 4   Segment             object 
 5   회원여부_이용가능           int64  
 6   회원여부_이용가능_CA        int64  
 7   회원여부_이용가능_카드론       int64  
 8   소지여부_신용             int64  
 9   소지카드수_유효_신용         int64  
 10  소지카드수_이용가능_신용       int64  
 11  입회일자_신용             int64  
 12  입회경과개월수_신용          int64  
 13  회원여부_연체             int64  
 14  이용거절여부_카드론          int64  
 15  동의여부_한도증액안내         int64  
 16  수신거부여부_TM           int64  
 17  수신거부여부_DM           int64  
 18  수신거부여부_메일           int64  
 19  수신거부여부_SMS          int64  
 20  가입통신회사코드            object 
 21  탈회횟수_누적             int64  
 22  최종탈회후경과월            int64  
 23  탈회횟수_발급6개월이내        int64  
 24  탈회횟수_발급1년이내         int6

In [5]:
cols_to_keep = [
    '이용금액_R3M_신용체크',
    '이용금액_R3M_신용',
    '_1순위카드이용금액',
    '_2순위카드이용금액',
    '기준년월',
    'ID',
    'Segment'
]

df1 = df1[cols_to_keep]
df1.info() # 240만개 행 * 7개 열

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400000 entries, 0 to 2399999
Data columns (total 7 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   이용금액_R3M_신용체크  int64 
 1   이용금액_R3M_신용    int64 
 2   _1순위카드이용금액     int64 
 3   _2순위카드이용금액     int64 
 4   기준년월           int64 
 5   ID             object
 6   Segment        object
dtypes: int64(5), object(2)
memory usage: 128.2+ MB


In [6]:
df1['ID'].nunique()

400000

In [7]:
df307 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/3.승인매출정보/201807_train_승인매출정보.parquet')
df308 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/3.승인매출정보/201808_train_승인매출정보.parquet')
df309 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/3.승인매출정보/201809_train_승인매출정보.parquet')
df310 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/3.승인매출정보/201810_train_승인매출정보.parquet')
df311 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/3.승인매출정보/201811_train_승인매출정보.parquet')
df312 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/3.승인매출정보/201812_train_승인매출정보.parquet')

In [8]:
# 3번 데이터 병합
df3 = pd.concat([df307, df308, df309, df310, df311, df312], ignore_index=True)
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400000 entries, 0 to 2399999
Columns: 406 entries, 기준년월 to 이용금액대
dtypes: float64(3), int64(385), object(18)
memory usage: 7.3+ GB


In [9]:
cols_to_keep = [
    '정상청구원금_B0M',
    '정상청구원금_B2M',
    '이용금액_일시불_R12M',
    '이용금액_일시불_B0M',
    '이용금액_일시불_R6M',
    '이용금액_일시불_R3M',
    '이용금액_오프라인_B0M',
    '정상입금원금_B5M',
    '정상입금원금_B0M',
    '이용금액_오프라인_R3M',
    '이용금액_오프라인_R6M',
    '정상입금원금_B2M',
    '_3순위업종_이용금액',
    '_2순위업종_이용금액',
    '_2순위쇼핑업종_이용금액',
    '최대이용금액_일시불_R12M',
    '이용금액대',
    '이용건수_신용_R12M',
    '이용건수_신판_R12M',
    '이용건수_일시불_R12M',
    '_1순위업종_이용금액',
    '_3순위쇼핑업종_이용금액',
    '이용건수_오프라인_B0M',
    '이용가맹점수',
    '쇼핑_도소매_이용금액',
    '이용건수_오프라인_R6M',
    '이용건수_오프라인_R3M',
    '이용건수_신용_B0M',
    '이용건수_신용_R6M',
    '이용건수_신용_R3M',
    '이용건수_신판_B0M',
    '이용건수_신판_R6M',
    '이용건수_일시불_B0M',
    '이용건수_신판_R3M',
    '이용건수_일시불_R6M',
    '이용건수_일시불_R3M',
    '_1순위교통업종_이용금액',
    '연체입금원금_B0M',
    '기준년월',
    'ID'
]
df3 = df3[cols_to_keep]
df3.head()


,정상청구원금_B0M,정상청구원금_B2M,이용금액_일시불_R12M,이용금액_일시불_B0M,이용금액_일시불_R6M,이용금액_일시불_R3M,이용금액_오프라인_B0M,정상입금원금_B5M,정상입금원금_B0M,이용금액_오프라인_R3M,...,이용건수_신판_B0M,이용건수_신판_R6M,이용건수_일시불_B0M,이용건수_신판_R3M,이용건수_일시불_R6M,이용건수_일시불_R3M,_1순위교통업종_이용금액,연체입금원금_B0M,기준년월,ID
0,14440,16524,20667,1995,8380,2877,4043,9205,6335,4131,...,9,67,8,22,64,21,200,8104,201807,TRAIN_000000
1,6024,2420,54341,2862,19426,14316,3980,2546,5198,14339,...,13,81,13,43,81,43,1215,826,201807,TRAIN_000001
2,21929,21826,55656,5854,35136,15694,4524,16949,12564,12250,...,10,56,10,33,54,31,1362,9364,201807,TRAIN_000002
3,18563,19172,10753,387,-7031,-2710,3975,8418,7639,10408,...,4,32,3,16,26,13,208,10923,201807,TRAIN_000003
4,0,272,-2129,0,957,-3383,0,0,0,0,...,-2,-2,-2,0,-2,0,0,0,201807,TRAIN_000004


In [10]:
df3['ID'].nunique() # 40만개

400000

In [11]:
df407 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/4.청구입금정보/201807_train_청구정보.parquet')
df408 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/4.청구입금정보/201808_train_청구정보.parquet')
df409 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/4.청구입금정보/201809_train_청구정보.parquet')
df410 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/4.청구입금정보/201810_train_청구정보.parquet')
df411 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/4.청구입금정보/201811_train_청구정보.parquet')
df412 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/4.청구입금정보/201812_train_청구정보.parquet')

df407.head()

,기준년월,ID,대표결제일,대표결제방법코드,대표청구지고객주소구분코드,대표청구서수령지구분코드,청구서수령방법,청구서발송여부_B0,청구서발송여부_R3M,청구서발송여부_R6M,...,할인금액_청구서_B0M,상환개월수_결제일_R6M,상환개월수_결제일_R3M,선결제건수_R6M,선결제건수_R3M,연체건수_R6M,연체건수_R3M,혜택수혜금액_R3M,포인트_마일리지_환산_B0M,혜택수혜금액
0,201807,TRAIN_000000,27,자동이체,미확인,당사페이앱+이메일,문자메세지,1,1,1,...,0,5,3,0,0,1,0,3,0,0
1,201807,TRAIN_000001,13,자동이체,주거지,우편,우편,1,1,1,...,163,6,3,0,0,0,0,0,0,0
2,201807,TRAIN_000002,1,자동이체,미확인,이메일,이메일,1,1,1,...,0,6,3,0,0,0,0,121,0,50
3,201807,TRAIN_000003,5,자동이체,주거지,우편,우편,1,1,1,...,0,5,3,2,0,0,0,3,0,2
4,201807,TRAIN_000004,13,자동이체,주거지,우편,우편,0,1,1,...,0,1,1,0,0,0,0,0,0,0


In [12]:
# 4번 데이터 병합
df4 = pd.concat([df407, df408, df409, df410, df411, df412], ignore_index=True)
df4

,기준년월,ID,대표결제일,대표결제방법코드,대표청구지고객주소구분코드,대표청구서수령지구분코드,청구서수령방법,청구서발송여부_B0,청구서발송여부_R3M,청구서발송여부_R6M,...,할인금액_청구서_B0M,상환개월수_결제일_R6M,상환개월수_결제일_R3M,선결제건수_R6M,선결제건수_R3M,연체건수_R6M,연체건수_R3M,혜택수혜금액_R3M,포인트_마일리지_환산_B0M,혜택수혜금액
0,201807,TRAIN_000000,27,자동이체,미확인,당사페이앱+이메일,문자메세지,1,1,1,...,0,5,3,0,0,1,0,3,0,0
1,201807,TRAIN_000001,13,자동이체,주거지,우편,우편,1,1,1,...,163,6,3,0,0,0,0,0,0,0
2,201807,TRAIN_000002,1,자동이체,미확인,이메일,이메일,1,1,1,...,0,6,3,0,0,0,0,121,0,50
3,201807,TRAIN_000003,5,자동이체,주거지,우편,우편,1,1,1,...,0,5,3,2,0,0,0,3,0,2
4,201807,TRAIN_000004,13,자동이체,주거지,우편,우편,0,1,1,...,0,1,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201812,TRAIN_399995,25,자동이체,주거지,우편,우편,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2399996,201812,TRAIN_399996,20,자동이체,미확인,이메일,이메일,1,1,1,...,0,6,3,0,0,0,0,164,0,53
2399997,201812,TRAIN_399997,20,자동이체,회사,우편,우편,1,1,1,...,0,6,3,0,0,0,0,0,0,0
2399998,201812,TRAIN_399998,20,자동이체,주거지,우편,우편,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
cols_to_keep = [
    '청구금액_R6M',
    '청구금액_R3M',
    '청구금액_B0',
    '기준년월',
    'ID'
]

df4 = df4[cols_to_keep]
df4.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400000 entries, 0 to 2399999
Data columns (total 5 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   청구금액_R6M  int64 
 1   청구금액_R3M  int64 
 2   청구금액_B0   int64 
 3   기준년월      int64 
 4   ID        object
dtypes: int64(4), object(1)
memory usage: 91.6+ MB


In [14]:
df4['ID'].nunique() # 40만개

400000

In [15]:
df507 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/5.잔액정보/201807_train_잔액정보.parquet')
df508 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/5.잔액정보/201808_train_잔액정보.parquet')
df509 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/5.잔액정보/201809_train_잔액정보.parquet')
df510 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/5.잔액정보/201810_train_잔액정보.parquet')
df511 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/5.잔액정보/201811_train_잔액정보.parquet')
df512 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/train/5.잔액정보/201812_train_잔액정보.parquet')

df507.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 82 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   기준년월               400000 non-null  int64  
 1   ID                 400000 non-null  object 
 2   잔액_일시불_B0M         400000 non-null  int64  
 3   잔액_할부_B0M          400000 non-null  int64  
 4   잔액_현금서비스_B0M       400000 non-null  int64  
 5   잔액_리볼빙일시불이월_B0M    400000 non-null  int64  
 6   잔액_리볼빙CA이월_B0M     400000 non-null  int64  
 7   잔액_카드론_B0M         400000 non-null  int64  
 8   월중평잔_일시불_B0M       400000 non-null  int64  
 9   월중평잔_할부_B0M        400000 non-null  int64  
 10  월중평잔_CA_B0M        400000 non-null  int64  
 11  카드론잔액_최종경과월        400000 non-null  int64  
 12  연체일자_B0M           1700 non-null    float64
 13  연체잔액_B0M           400000 non-null  int64  
 14  연체잔액_일시불_B0M       400000 non-null  int64  
 15  연체잔액_할부_B0M        400000 non-null  int64  
 16  연체

In [16]:
# 5번 데이터 병합
df5 = pd.concat([df507, df508, df509, df510, df511, df512], ignore_index=True)


cols_to_keep = [
    '평잔_일시불_6M',
    '월중평잔_일시불_B0M',
    '월중평잔_일시불',
    '평잔_일시불_3M',
    '잔액_할부_B0M',
    '기준년월',
    'ID'
]

df5 = df5[cols_to_keep]
df5.head()


,평잔_일시불_6M,월중평잔_일시불_B0M,월중평잔_일시불,평잔_일시불_3M,잔액_할부_B0M,기준년월,ID
0,2440,1084,1503,1791,962,201807,TRAIN_000000
1,2677,4090,4447,3761,2390,201807,TRAIN_000001
2,9118,5006,5540,6796,5113,201807,TRAIN_000002
3,884,487,606,772,5025,201807,TRAIN_000003
4,21,0,0,0,0,201807,TRAIN_000004


In [17]:
# 현재 코드에 이어붙입니다.

# --- 데이터 병합 (df1, df3, df4, df5를 ID와 기준년월 기준으로) ---
print("\n--- 모든 데이터프레임 ID와 기준년월 기준으로 병합 시작 ---")

# 첫 번째 병합: df1과 df3 병합
# df1의 7개 컬럼과 df3의 40개 컬럼이 합쳐집니다.
# 'ID'와 '기준년월'은 양쪽에 모두 있으므로, 이를 기준으로 병합합니다.
merged_df = pd.merge(df1, df3, on=['ID', '기준년월'], how='inner')
print(f"df1과 df3 병합 완료. 현재 행 수: {len(merged_df)}, 컬럼 수: {len(merged_df.columns)}")
print(merged_df.head())

# 두 번째 병합: merged_df와 df4 병합
# df4의 5개 컬럼이 추가됩니다.
merged_df = pd.merge(merged_df, df4, on=['ID', '기준년월'], how='inner')
print(f"merged_df와 df4 병합 완료. 현재 행 수: {len(merged_df)}, 컬럼 수: {len(merged_df.columns)}")
print(merged_df.head())


# 세 번째 병합: merged_df와 df5 병합
# df5의 7개 컬럼이 추가됩니다.
merged_df = pd.merge(merged_df, df5, on=['ID', '기준년월'], how='inner')
print(f"merged_df와 df5 병합 완료. 현재 행 수: {len(merged_df)}, 컬럼 수: {len(merged_df.columns)}")
print(merged_df.head())

# 최종 병합된 데이터프레임 정보 확인
print("\n--- 최종 병합된 데이터프레임 (merged_df) 정보 ---")
merged_df.info()

# 메모리 최적화를 위해 사용하지 않는 개별 df 삭제
del df1, df3, df4, df5, df107, df108, df109, df110, df111, df112, \
    df307, df308, df309, df310, df311, df312, \
    df407, df408, df409, df410, df411, df412, \
    df507, df508, df509, df510, df511, df512
gc.collect() # 가비지 컬렉션 실행

print("\n--- 데이터 병합 및 메모리 정리 완료. ---")


--- 모든 데이터프레임 ID와 기준년월 기준으로 병합 시작 ---
df1과 df3 병합 완료. 현재 행 수: 2400000, 컬럼 수: 45
   이용금액_R3M_신용체크  이용금액_R3M_신용  _1순위카드이용금액  _2순위카드이용금액    기준년월            ID  \
0            196          196        3681           0  201807  TRAIN_000000   
1          13475        13475       13323           0  201807  TRAIN_000001   
2          23988        23988       24493           0  201807  TRAIN_000002   
3           3904         3904        5933           0  201807  TRAIN_000003   
4           1190            0           0           0  201807  TRAIN_000004   

  Segment  정상청구원금_B0M  정상청구원금_B2M  이용금액_일시불_R12M  ...  이용건수_신용_R6M  \
0       D       14440       16524          20667  ...           77   
1       E        6024        2420          54341  ...           81   
2       C       21929       21826          55656  ...           62   
3       D       18563       19172          10753  ...           42   
4       E           0         272          -2129  ...           -2   

   이용건수_신용_R3M  이용건수_신판

In [18]:
merged_df.info()
merged_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400000 entries, 0 to 2399999
Data columns (total 53 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   이용금액_R3M_신용체크    int64 
 1   이용금액_R3M_신용      int64 
 2   _1순위카드이용금액       int64 
 3   _2순위카드이용금액       int64 
 4   기준년월             int64 
 5   ID               object
 6   Segment          object
 7   정상청구원금_B0M       int64 
 8   정상청구원금_B2M       int64 
 9   이용금액_일시불_R12M    int64 
 10  이용금액_일시불_B0M     int64 
 11  이용금액_일시불_R6M     int64 
 12  이용금액_일시불_R3M     int64 
 13  이용금액_오프라인_B0M    int64 
 14  정상입금원금_B5M       int64 
 15  정상입금원금_B0M       int64 
 16  이용금액_오프라인_R3M    int64 
 17  이용금액_오프라인_R6M    int64 
 18  정상입금원금_B2M       int64 
 19  _3순위업종_이용금액      int64 
 20  _2순위업종_이용금액      int64 
 21  _2순위쇼핑업종_이용금액    int64 
 22  최대이용금액_일시불_R12M  int64 
 23  이용금액대            object
 24  이용건수_신용_R12M     int64 
 25  이용건수_신판_R12M     int64 
 26  이용건수_일시불_R12M    int64 
 27  _1순위업종_이용금액      int64 
 28  _3순위쇼핑업종_이용금

,이용금액_R3M_신용체크,이용금액_R3M_신용,_1순위카드이용금액,_2순위카드이용금액,기준년월,ID,Segment,정상청구원금_B0M,정상청구원금_B2M,이용금액_일시불_R12M,...,_1순위교통업종_이용금액,연체입금원금_B0M,청구금액_R6M,청구금액_R3M,청구금액_B0,평잔_일시불_6M,월중평잔_일시불_B0M,월중평잔_일시불,평잔_일시불_3M,잔액_할부_B0M
0,196,196,3681,0,201807,TRAIN_000000,D,14440,16524,20667,...,200,8104,88693,46588,12226,2440,1084,1503,1791,962
1,13475,13475,13323,0,201807,TRAIN_000001,E,6024,2420,54341,...,1215,826,16861,10530,5834,2677,4090,4447,3761,2390
2,23988,23988,24493,0,201807,TRAIN_000002,C,21929,21826,55656,...,1362,9364,165221,85931,21866,9118,5006,5540,6796,5113
3,3904,3904,5933,0,201807,TRAIN_000003,D,18563,19172,10753,...,208,10923,127371,61518,16356,884,487,606,772,5025
4,1190,0,0,0,201807,TRAIN_000004,E,0,272,-2129,...,0,0,155,0,0,21,0,0,0,0


In [21]:

# 저장할 파일 경로 설정 (구글 드라이브 내 특정 폴더에 저장하는 것을 권장)
# 예시 경로: '/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/r05_merged_data.parquet'
# 실제 경로를 맞춰주세요.
output_file_path = '/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/r05_train.parquet'

# Parquet 파일로 저장
# index=False: 데이터프레임의 인덱스를 파일에 저장하지 않습니다.
#              대부분의 경우 인덱스는 특성으로 사용되지 않으므로 저장 공간을 절약하고 불필요한 컬럼 생성을 막습니다.
merged_df.to_parquet(output_file_path, index=False)

print(f"merged_df가 '{output_file_path}' 경로에 성공적으로 저장되었습니다.")

merged_df가 '/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/r05_train.parquet' 경로에 성공적으로 저장되었습니다.


### test 데이터

In [25]:
# 1. 테스트 회원정보 데이터 로드 및 병합 (train 데이터 df1과 유사)
test_df101 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/test/1.회원정보/201807_test_회원정보.parquet')
test_df102 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이타 분석/12_파이널 프로젝트/data/test/1.회원정보/201808_test_회원정보.parquet')
test_df103 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/test/1.회원정보/201809_test_회원정보.parquet')
test_df104 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/test/1.회원정보/201810_test_회원정보.parquet')
test_df105 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/test/1.회원정보/201811_test_회원정보.parquet')
test_df106 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/test/1.회원정보/201812_test_회원정보.parquet')


test_df1 = pd.concat([test_df101, test_df102, test_df103, test_df104,
                      test_df105, test_df106], ignore_index=True)

# 훈련 데이터 df1과 동일한 컬럼만 유지
cols_to_keep_df1 = [
    '이용금액_R3M_신용체크',
    '이용금액_R3M_신용',
    '_1순위카드이용금액',
    '_2순위카드이용금액',
    '기준년월',
    'ID' # 테스트 데이터에는 Segment 컬럼이 없음
]
test_df1 = test_df1[cols_to_keep_df1]
print(f"테스트 회원정보 (test_df1) 병합 완료. 행: {len(test_df1)}, 열: {len(test_df1.columns)}")
display(test_df1.head())


# 2. 테스트 승인매출정보 데이터 로드 및 병합
test_df301 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/test/3.승인매출정보/201807_test_승인매출정보.parquet')
test_df302 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/test/3.승인매출정보/201808_test_승인매출정보.parquet')
test_df303 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/test/3.승인매출정보/201809_test_승인매출정보.parquet')
test_df304 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/test/3.승인매출정보/201810_test_승인매출정보.parquet')
test_df305 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/test/3.승인매출정보/201811_test_승인매출정보.parquet')
test_df306 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/test/3.승인매출정보/201812_test_승인매출정보.parquet')


test_df3 = pd.concat([test_df301, test_df302, test_df303, test_df304,
                      test_df305, test_df306], ignore_index=True)

# 훈련 데이터 df3과 동일한 컬럼만 유지
cols_to_keep_df3 = [
    '정상청구원금_B0M', '정상청구원금_B2M', '이용금액_일시불_R12M', '이용금액_일시불_B0M',
    '이용금액_일시불_R6M', '이용금액_일시불_R3M', '이용금액_오프라인_B0M', '정상입금원금_B5M',
    '정상입금원금_B0M', '이용금액_오프라인_R3M', '이용금액_오프라인_R6M', '정상입금원금_B2M',
    '_3순위업종_이용금액', '_2순위업종_이용금액', '_2순위쇼핑업종_이용금액', '최대이용금액_일시불_R12M',
    '이용금액대', '이용건수_신용_R12M', '이용건수_신판_R12M', '이용건수_일시불_R12M',
    '_1순위업종_이용금액', '_3순위쇼핑업종_이용금액', '이용건수_오프라인_B0M', '이용가맹점수',
    '쇼핑_도소매_이용금액', '이용건수_오프라인_R6M', '이용건수_오프라인_R3M', '이용건수_신용_B0M',
    '이용건수_신용_R6M', '이용건수_신용_R3M', '이용건수_신판_B0M', '이용건수_신판_R6M',
    '이용건수_일시불_B0M', '이용건수_신판_R3M', '이용건수_일시불_R6M', '이용건수_일시불_R3M',
    '_1순위교통업종_이용금액', '연체입금원금_B0M',
    '기준년월', 'ID'
]
test_df3 = test_df3[cols_to_keep_df3]
print(f"테스트 승인매출정보 (test_df3) 병합 완료. 행: {len(test_df3)}, 열: {len(test_df3.columns)}")
display(test_df3.head())

# 3. 테스트 청구입금정보 데이터 로드 및 병합
test_df401 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/test/4.청구입금정보/201807_test_청구정보.parquet')
test_df402 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/test/4.청구입금정보/201808_test_청구정보.parquet')
test_df403 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/test/4.청구입금정보/201809_test_청구정보.parquet')
test_df404 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/test/4.청구입금정보/201810_test_청구정보.parquet')
test_df405 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/test/4.청구입금정보/201811_test_청구정보.parquet')
test_df406 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/test/4.청구입금정보/201812_test_청구정보.parquet')

test_df4 = pd.concat([test_df401, test_df402, test_df403, test_df404,
                      test_df405, test_df406], ignore_index=True)

# 훈련 데이터 df4와 동일한 컬럼만 유지
cols_to_keep_df4 = [
    '청구금액_R6M',
    '청구금액_R3M',
    '청구금액_B0',
    '기준년월',
    'ID'
]
test_df4 = test_df4[cols_to_keep_df4]
print(f"테스트 청구입금정보 (test_df4) 병합 완료. 행: {len(test_df4)}, 열: {len(test_df4.columns)}")
display(test_df4.head())


# 4. 테스트 잔액정보 데이터 로드 및 병합
test_df501 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/test/5.잔액정보/201807_test_잔액정보.parquet')
test_df502 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/test/5.잔액정보/201808_test_잔액정보.parquet')
test_df503 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/test/5.잔액정보/201809_test_잔액정보.parquet')
test_df504 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/test/5.잔액정보/201810_test_잔액정보.parquet')
test_df505 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/test/5.잔액정보/201811_test_잔액정보.parquet')
test_df506 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/data/test/5.잔액정보/201812_test_잔액정보.parquet')

test_df5 = pd.concat([test_df501, test_df502, test_df503, test_df504,
                      test_df505, test_df506], ignore_index=True)

# 훈련 데이터 df5와 동일한 컬럼만 유지
cols_to_keep_df5 = [
    '평잔_일시불_6M',
    '월중평잔_일시불_B0M',
    '월중평잔_일시불',
    '평잔_일시불_3M',
    '잔액_할부_B0M',
    '기준년월',
    'ID'
]
test_df5 = test_df5[cols_to_keep_df5]
print(f"테스트 잔액정보 (test_df5) 병합 완료. 행: {len(test_df5)}, 열: {len(test_df5.columns)}")
display(test_df5.head())

# 각 test_dfX의 ID 고유값 개수 확인 (40만개 * 8개월 = 320만 행, 고유 ID 40만개)
print(f"\ntest_df1 고유 ID 개수: {test_df1['ID'].nunique()}")
print(f"test_df3 고유 ID 개수: {test_df3['ID'].nunique()}")
print(f"test_df4 고유 ID 개수: {test_df4['ID'].nunique()}")
print(f"test_df5 고유 ID 개수: {test_df5['ID'].nunique()}")


# 첫 번째 병합: test_df1과 test_df3 병합
merged_test_df = pd.merge(test_df1, test_df3, on=['ID', '기준년월'], how='inner')
print(f"test_df1과 test_df3 병합 완료. 현재 행 수: {len(merged_test_df)}, 컬럼 수: {len(merged_test_df.columns)}")

# 두 번째 병합: merged_test_df와 test_df4 병합
merged_test_df = pd.merge(merged_test_df, test_df4, on=['ID', '기준년월'], how='inner')
print(f"merged_test_df와 test_df4 병합 완료. 현재 행 수: {len(merged_test_df)}, 컬럼 수: {len(merged_test_df.columns)}")

# 세 번째 병합: merged_test_df와 test_df5 병합
merged_test_df = pd.merge(merged_test_df, test_df5, on=['ID', '기준년월'], how='inner')
print(f"merged_test_df와 test_df5 병합 완료. 현재 행 수: {len(merged_test_df)}, 컬럼 수: {len(merged_test_df.columns)}")

# 최종 병합된 테스트 데이터프레임 정보 확인
print("\n--- 최종 병합된 테스트 데이터프레임 (merged_test_df) 정보 ---")
merged_test_df.info()
display(merged_test_df.head())

# 메모리 최적화를 위해 사용하지 않는 개별 df 삭제
del test_df1, test_df3, test_df4, test_df5
del test_df101, test_df102, test_df103, test_df104, test_df105, test_df106
del test_df301, test_df302, test_df303, test_df304, test_df305, test_df306
del test_df401, test_df402, test_df403, test_df404, test_df405, test_df406
del test_df501, test_df502, test_df503, test_df504, test_df505, test_df506
gc.collect()

print("\n--- 테스트 데이터 병합 및 메모리 정리 완료. ---")

# 저장할 파일 경로 설정 (구글 드라이브 내 특정 폴더에 저장하는 것을 권장)
# 예시 경로: '/content/drive/MyDrive/r05_test.parquet'
# 실제 경로를 맞춰주세요.
output_test_file_path = '/content/drive/MyDrive/r05_test.parquet'
merged_test_df.to_parquet(output_test_file_path, index=False)
print(f"merged_test_df가 '{output_test_file_path}' 경로에 성공적으로 저장되었습니다.")

테스트 회원정보 (test_df1) 병합 완료. 행: 600000, 열: 6
   이용금액_R3M_신용체크  이용금액_R3M_신용  _1순위카드이용금액  _2순위카드이용금액    기준년월          ID
0          21458        21458       13852        6021  201807  TEST_00000
1          18681        10759       11065           0  201807  TEST_00001
2          40758        40758       27071       10704  201807  TEST_00002
3           5255         5255        4827           0  201807  TEST_00003
4          16148        14290        8011        6387  201807  TEST_00004
테스트 승인매출정보 (test_df3) 병합 완료. 행: 600000, 열: 40
   정상청구원금_B0M  정상청구원금_B2M  이용금액_일시불_R12M  이용금액_일시불_B0M  이용금액_일시불_R6M  \
0        6495        2012          49063          7650         26595   
1        9644        5273           7771           367         -7578   
2       11519       11366          73003          8547         48483   
3        1375        1072           7421          1270           403   
4        3951        1618          10493           709          1238   

   이용금액_일시불_R3M  이용금액_오프라인_B0M  정상